In [1]:
import h5py
import numpy 
import pylab as mplot
%pylab inline
import astropy.io.fits as pf
import healpy as hp

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [12]:
original = pf.open('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/des-y1/DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits')
original[1].header



XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional binary table                     
NAXIS1  =                   24 / width of table in bytes                        
NAXIS2  =             51843669 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group (required keyword)              
TFIELDS =                    3 / number of fields in each row                   
TTYPE1  = 'RA      '           / label for field   1                            
TFORM1  = 'D       '           / data format of field: 8-byte DOUBLE            
TTYPE2  = 'DEC     '           / label for field   2                            
TFORM2  = 'D       '           / data format of field: 8-byte DOUBLE            
TTYPE3  = 'Z       '        

In [31]:
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/star-challenge/cosmodc2/TXPipe-full-output/random_cats.hdf5') as f:
    print(np.array(f['randoms/bin']))
    print(np.array(f['randoms/comoving_distance']))
    print(f['randoms'].keys())
    print(f.keys())
    print(dict(f['randoms'].attrs))
    


[0 0 0 ... 4 4 4]
[1721.92885605 1133.18173043 1084.83263351 ... 3593.61871468 3731.27138923
 3317.47376116]
<KeysViewHDF5 ['bin', 'comoving_distance', 'dec', 'ra', 'z']>
<KeysViewHDF5 ['provenance', 'randoms']>
{}


In [33]:
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/star-challenge/cosmodc2/TXPipe-full-output/binned_random_catalog.hdf5') as f:
    print(f['randoms'].keys())
    print(f['randoms/bin_0'].keys())
    print(dict(f['randoms/bin_0'].attrs))



<KeysViewHDF5 ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']>
<KeysViewHDF5 ['comoving_distance', 'dec', 'ra', 'z']>
{}


In [ ]:
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/des-y1/randoms_desy1_RM.h5', 'w') as f:

    f.create_group("provenance")
    f.create_group("randoms")

    f['randoms/ra'] = original[1].data['ra'].copy()
    f['randoms/dec'] = original[1].data['dec'].copy()
    f['randoms/z'] = original[1].data['z'].copy()
    f['randoms/comoving_distance'] = cosmo.comoving_distance(original[1].data['z']).value
    
    bin_array = np.zeros(len(original[1].data['ra'])) - 1
    mask = (original[1].data['z']>=0.15)*(original[1].data['z']<0.3)
    bin_array[mask] = 0
    mask = (original[1].data['z']>=0.3)*(original[1].data['z']<0.45)
    bin_array[mask] = 1
    mask = (original[1].data['z']>=0.45)*(original[1].data['z']<0.6)
    bin_array[mask] = 2
    mask = (original[1].data['z']>=0.6)*(original[1].data['z']<0.75)
    bin_array[mask] = 3
    mask = (original[1].data['z']>=0.75)*(original[1].data['z']<0.9)
    bin_array[mask] = 4
    f['randoms/bin'] = bin_array
    
    

    
    

In [21]:
import astropy
from astropy.cosmology import Planck18 as cosmo
cosmo.comoving_distance(0.1).value


432.5656977383524

In [26]:
# dir(astropy.cosmology)

In [42]:
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/des-y1/randoms_desy1_RM.h5') as f1:
    with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/des-y1/binned_randoms_desy1_RM.h5', 'w') as f2:
        
        f2.create_group("provenance")
        f2.create_group("randoms")
        f2.create_group("randoms/bin_0")
        f2.create_group("randoms/bin_1")
        f2.create_group("randoms/bin_2")    
        f2.create_group("randoms/bin_3") 
        f2.create_group("randoms/bin_4")

        for i in range(5):
            mask = (f1['randoms/bin'][:]==i)
            f2['randoms/bin_'+str(i)+'/ra'] = f1['randoms/ra'][mask]
            f2['randoms/bin_'+str(i)+'/dec'] = f1['randoms/dec'][mask]
            f2['randoms/bin_'+str(i)+'/z'] = f1['randoms/z'][mask]
            f2['randoms/bin_'+str(i)+'/comoving_distance'] = f1['randoms/comoving_distance'][mask]

        

In [43]:
import treecorr

In [45]:
config = { "ext": "randoms",
            "ra_col": "ra",
            "dec_col": "dec",
            "ra_units": "degree",
            "dec_units": "degree",
            "every_nth": 100,
            "npatch": 10,
        }

        # Create the catalog
# cat = treecorr.Catalog('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/des-y1/randoms_desy1_RM.h5', config)
cat = treecorr.Catalog('/global/cfs/cdirs/lsst/groups/WL/projects/star-challenge/cosmodc2/TXPipe-full-output/binned_random_catalog.hdf5', config)

